#Company bankruptcy

In [1]:
# jeśli kod uruchamiany na Colab
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    import os
    os.chdir('/content/drive/MyDrive/Colab Notebooks/DW/master_class/company_bankruptcy_prediction/notebooks')
    import sys
    !pip install eli5;
    !pip install scikit-plot;
    !pip install catboost;


Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.2/216.2 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for eli5: filename=eli5-0.13.0-py2.py3-none-any.whl size=107717 sha256=210a232df350c741c1d0feb5dabd27f551de0368bd560b7c29333a500b9630b7
  Stored in directory: /root/.cache/pip/wheels/b8/58/ef/2cf4c306898c2338d51540e0922c8e0d6028e07007085c0004
Successfully built eli5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 2.9 MB/s eta 0:00:00


## Wczytywanie bibliotek

In [2]:
import pandas as pd
import numpy as np
np.random.seed(0)

import catboost as ctb
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import f1_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, StackingClassifier

from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
import eli5
from eli5.sklearn import PermutationImportance

import scikitplot as skplt

import matplotlib.pyplot as plt

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import ComplementNB
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, cross_val_predict
from itertools import combinations



from tqdm.notebook import tqdm

## Wczytywanie danych

In [3]:
df_train = pd.read_hdf("../input/train_taiwan.h5")
df_test = pd.read_hdf("../input/test_taiwan.h5")

df_all = pd.concat([df_train, df_test]).reset_index(drop=True)
df_all.columns = [x.replace(" ", "_").lower() for x in df_all.columns]

df_train.shape, df_test.shape, df_all.shape

((3409, 97), (3410, 96), (6819, 97))

In [4]:
df_answers=pd.read_csv("../input/answer.csv")
#plik z odpowiedzami do pliku test_taiwan

In [5]:
df_answers.head(10)

,id,target
0,1,0
1,2,0
2,3,0
3,5,0
4,7,0
5,11,0
6,12,0
7,13,0
8,14,0
9,16,0


## ZMIENNE GLOBALNE

### słownik models_trained

In [6]:
# w tym słowniku będę zbierał info o wytrenowanych modelach

"""

    LGBMClassifier: (słownik)
    - Method: count_positive_1
    - Feats_count: 30
    - F1: 0.5001178272652292
    - Feats: ['borrowing_dependency', 'retained_earnings/total_assets', 'quick_asset/current_liabilities', 'continuous_profit_rate_(after_tax)', 'permanent_net_profit_growth_rate', 'interest-bearing_debt_interest_rate', 'net_income_to_total_assets', 'book_value_per_share_(b)', 'total_assets_growth_rate_', 'cash_flow_ratio', 'roa(c)_before_tax_and_interest_before_depreciation', 'return_on_total_assets_growth_rate', 'research_and_development_expense_ratio', 'cash_/_current_liability', 'persistent_eps_in_the_last_four_seasons', 'equity_to_long-term_liability', 'net_worth_growth_rate', 'realized_sales_gross_profit_growth_rate', 'current_liability/equity', 'inventory_turnover_rate_(times)', 'turnover_per_share_(rmb)', 'net_value/asset', 'revenue_per_person', 'cash_flow_to_total_assets', 'total_income_/_total_expense', 'cash_flow_to_equity', 'equipment_rate_per_person', 'non-industry_revenue_and_expenditure/revenue', 'cash_flow_per_share', 'total_assets_to_gnp_price']
    - params: (słownik)
      - boosting_type: gbdt
      - class_weight: None
      - colsample_bytree: 1.0
      - importance_type: split
      - learning_rate: 0.1
      - max_depth: 5
      - min_child_samples: 20
      - min_child_weight: 0.001
      - min_split_gain: 0.0
      - n_estimators: 100
      - n_jobs: None
      - num_leaves: 31
      - objective: None
      - random_state: None
      - reg_alpha: 0.0
      - reg_lambda: 0.0
      - subsample: 1.0
      - subsample_for_bin: 200000
      - subsample_freq: 0
      - verbosity: -1
      - constructor: <class 'lightgbm.sklearn.LGBMClassifier'>

"""

models_trained={}


## Funkcje pomocnicze

In [7]:
def draw_white(draw):
        '''
        Zmiana koloru opisu osi na biały
        '''
        ax=draw
        ax.xaxis.label.set_color('white')
        ax.yaxis.label.set_color('white')
        ax.tick_params(colors='white', which='both')

### make_exepriment()

In [8]:
def make_experiment(model, feats=None, threshold=0.5, n_splits=3, black_list=["target"], show_feats=False, show_cr=False, show_cm=False, show_pr=False, show_lc=False,show_wg=False,show_f1=False,return_y_predict=False):
    '''
    Niestety skomplikowane funkcja która trenuje i waliduje modele
    na zadanych lub wyliczonych cechach (feats)

    INPUT :
    model - zainicjowany model do treningu
    feats - zbiór cech
    threshold  - próg prawdopodobieństwa od którego zaliczymy obserwację do klasy 1
    n_splits- podział zbioru treningowego
    black_list - zbiór cech wykluczonych
    show_feats - pokazuje cechy które zostaną użyte do treningu
    show_cr - pokazuje classification_report
    show_cm - pokazuje confusion matrix
    show_pr - pokazuje wykres precision_recall
    show_lc - pokazuje wykres krzywej uczenia
    show_wg - pokazuje cechy wagi z eli5.show_weights
    show_f1 - pokazuje metryke f1 liczoną jako średnia z poszczególnych splitów
    return_y_predict - zwraca y_predict (prawdopodobieństwa dla y)

    OUTPUT:
       JEŚLI  return_y_predict: zwraca krotke ( prawdopodobieństwa dla  y i
       miarę F1 liczoną jako średnia f1 dla poszczególnych splitów )
       ELSE  tylko miarę F1 liczoną jako średnia f1 dla poszczególnych splitów

    '''
    if feats is None:
        num_feats = df_all.select_dtypes("number").columns
        feats = [x for x in num_feats if x not in black_list]

    if show_feats:
        print(feats)

    df_train = df_all[ df_all["target"].notnull() ]
    X_train = df_train[feats].values
    y_train = df_train["target"].values

    scores = []
    y_pred_all= np.zeros((len(X_train), 2))
    cv=StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=0)
    for train_idx, test_idx in cv.split(X_train, y_train):

        model.fit(X_train[train_idx], y_train[train_idx])

        y_probas = model.predict_proba(X_train[test_idx])

        y_pred = (y_probas[:,1] > threshold).astype(int)
        y_pred_all[test_idx,1]=y_pred


        if show_cr:
            print(classification_report(y_train[test_idx], y_pred))

        if show_cm:
            draw_white(skplt.metrics.plot_confusion_matrix(y_train[test_idx], y_pred))#normalize=True


        if show_pr:
            draw_white(skplt.metrics.plot_precision_recall(y_train[test_idx], y_probas))


        score = f1_score(y_train[test_idx], y_pred)
        scores.append(score)


    if show_f1:
      print(f"F1 score:{round(np.mean(scores),5)}, std:{round(np.std(scores),5)} ")

    if show_lc:
        draw_white(skplt.estimators.plot_learning_curve(model, X_train, y_train, cv=cv, scoring="f1", random_state=0))

    if show_wg:
        eli5.show_weights(model, feature_names=feats, top=50)

    if return_y_predict: return (y_pred_all,np.mean(scores))
    else: return np.mean(scores)


def my_hist(df_all, feat_name, left_perc=1, right_perc=99):
    min_value = np.percentile( df_all[feat_name], 1)
    max_value = np.percentile( df_all[feat_name], 99)

    (
        df_all[
            (df_all[feat_name] > min_value) &
            (df_all[feat_name] < max_value)
        ][feat_name]
    ).hist(bins=100)
    plt.title("{}\n, left_perc={}, right_perc={}".format(feat_name, left_perc, right_perc))
    plt.show()





### save_result()

In [9]:

def save_result(filename, model, threshold=0.1,feats=[],save=True):

    '''
    Funkcja trenuje model na zadanych cechach i zadanym progu prawdopodobieństwa.
    A nastene wykonuje predykcje dla zbioru testowego dla zadanego threshold
    Jeśłi parametr save= True to dodatkowo zapisuje predykcje do pliku
    filename w katalgu ouptput


    '''
    df_train = df_all[ df_all["target"].notnull() ]
    df_test = df_all[ df_all["target"].isnull() ].copy()

    if not feats:
      num_feats = df_all.select_dtypes("number").columns
      feats = [x for x in num_feats if x not in ["target"]]


    model.fit(df_train[feats].values, df_train["target"].values)

    df_test["target"] = (model.predict_proba(df_test[feats].values)[:,1] > threshold).astype(int)
    if save:
      output_file = "../output/{}".format(filename)
      df_test[ ["id", "target"] ].to_csv(output_file, index=False)
      print("Saved: ", output_file)
    if save==False : return  df_test[ ["id", "target"] ]

### model_predict()

In [10]:
def model_predict(model, feats=[],threshold=0.1):
    ''' funkcja zwraca predykcje modelu dla zbioru testowego'''
    return save_result('',model,feats=feats,threshold=threshold,save=False)
    # df_train = df_all[ df_all["target"].notnull() ]
    # df_test = df_all[ df_all["target"].isnull() ].copy()

    # if not feats:
    #   num_feats = df_all.select_dtypes("number").columns
    #   feats = [x for x in num_feats if x not in ["target"]]


    # model.fit(df_train[feats].values, df_train["target"].values)

    # df_test["target"] = (model.predict_proba(df_test[feats].values)[:,1] > 0.1).astype(int)

    # return df_test[ ["id", "target"] ].reset_index(drop=True)



### kaggle_result

In [11]:
def kaggle_result(y_test,y_pred):

    '''
    Funkcja liczy miary F1 dla zadanycy y_test i y_pred, dzielą zbiory
    wejściowe na dwie częsci public i private.
    Trochę symulacja działania Kaggle.

    '''

    # Połącz obie ramki danych
    df = pd.merge(y_test, y_pred, on="id")

    #df = df.rename(columns={"target_x": "y_test", "target_y": "y_pred"})
    df.rename(columns={df.columns[1]: 'y_test', df.columns[2]: 'y_pred'}, inplace=True)

    # Podziel dane na dwa zbiory, zachowując balans klas
    public, private = train_test_split(df, test_size=0.5, stratify=df['y_test'], random_state=42)

    # # Oblicz miarę F1 dla zbioru public
    f1_public = f1_score(public['y_test'], public['y_pred'], average='binary')

    # # Oblicz miarę F1 dla zbioru private
    f1_private = f1_score(private['y_test'],private['y_pred'], average='binary')

    return f1_public, f1_private



### model_mix_predict()

In [12]:
def generuj_ulamki(p):
    '''
    Funkcja dzieli 1 na p części, uzupełniając ostatnią cześć w taki sposób
    aby całość sumowała się do 1.  Przykład : dla p=3 funkcja zwróci listę:
    [0.33,0.33,0.34]
    '''
    u = [round(1/p, 2) for _ in range(p)]
    roznica = round(1 - sum(u), 2)

    if roznica != 0:
        u[-1] += roznica

    return u

In [13]:
def model_mix_predict(models,threshold=0.2,feats=[]):
  '''
  Funkcja łączy predykcje dla zbioru testowego wykonanego dla modeli w zależności
  od wartości threshold uwzględnia od 1 do len(models) jedynek ze wszystkich poszczególnych predykcji
  tzn jeśli  próg ustawiony na dwie jedynki to jedynka w zbiorze wynikowym pojawi się gdy w
  predykcjach poszczególnych modeli pojawią się min dwie jedynki

  input :
   models - lista z nazwami modlei
   threshsold - próg
   feats- nazawa klucza opoisującego cechy w słwoniku models_trained
  ouptput:
   df["id","target"]
  '''
  mnozniki=generuj_ulamki(len(models))

  merged_df = pd.DataFrame(columns=['id'])
  for i,key in enumerate(models) :

    m=models_trained[key]
    model_to_train=m['constructor'](**m['params'])
    if feats: feats_to_train=m[feats]
    else: feats_to_train=feats
    ml_predict=model_predict(model=model_to_train,feats=feats_to_train)
    target_column_name = f'{key}'
    merged_df[target_column_name] = mnozniki[i]*ml_predict['target']

  merged_df['id']=ml_predict['id']
  merged_df['target'] = (merged_df.drop('id',axis=1).sum(axis=1) >=threshold).astype(int)
  return merged_df[['id','target']]


### test_models_predict()

In [14]:


def test_models_predict(models,feats=[]):
  '''
  funkcja trenuje modele z listy models i cechami feats na zbiorze Treningowym
  zwraca tablice która zawiera predykcje ze wszystkich modeli wykonane na zbiorze testowym
  models: lista modeli
  feats: nazwa klucza z cechami

  df_predict : dataframe z predykcjami  o kolumnach id, 'nazwa modelu

  '''

  rows_count=df_all[ df_all["target"].isnull() ].shape[0] #wybór zbioru testowego
  meta_features = np.zeros((rows_count, len(models)))
  #print(f'meta shape {meta_featues.shape[0]}')

  for i,key in enumerate(models):
    mod=models_trained[key]['constructor'](**models_trained[key]['params'])
    if feats: feats=models_trained[key]['Feats']
    pr=model_predict(model=mod,feats=feats)

    meta_features[:, i] = pr['target']

  df_predict=pd.DataFrame(meta_features,columns=models)

  df_predict['id']=pr['id'].values
  return df_predict

In [15]:
df_all[ df_all["target"].isnull() ].shape[0]

3410

### train_models_predict()


In [16]:
def train_models_predict(models, threshold=0.1, feats=[]):
  '''
  funkcja trenuje i waliduje t modele z listy models i cechami feats na zbiorze treningowym
  zwraca tablice która zawiera predykcje ze wszystkich modeli,
  models: lista modeli
  feats: nazwa klucza z cechami

  df_predict : dataframe z predykcjami  o kolumnach id, 'nazwa modelu

  '''
  rows_count=df_all[ df_all["target"].notnull() ].shape[0]
  meta_features = np.zeros((rows_count, len(models)))

  for i, key in enumerate(models):
    # Generowanie predykcji za pomocą walidacji krzyżowej
    model=models_trained[key]['constructor'](**models_trained[key]['params'])
    if feats: predictions,f1 = make_experiment( model,threshold=0.1,feats=models_trained[key][feats], return_y_predict=True)
    else: predictions,f1 = make_experiment( model,threshold=0.1, return_y_predict=True)
   # predictions,f1 = make_experiment( model,threshold=0.1,feats=models_trained[key][feats], return_y_predict=True)

    meta_features[:, i] = predictions[:, 1]
    #print(f"F1 Score for {key}: {f1:.4f}")

  df_train = df_all[ df_all["target"].notnull() ]
  df_predict=pd.DataFrame(meta_features,columns=models)
  df_predict['id']=df_train['id']

  return df_predict




### mixed_models_predict()

In [17]:
def mixed_models_predict(pr,threshold,y_true='train',return_y_predict=False):
    '''
    Funkcja generuje łączną predykcję  dla wielu modeli których predykcje
    w dataframe pr["id","nazwa_model"....] dla zadanego progu threshold
    od której aktywuje jedynki
    Zwraca predykcję lub wprost miarę f1
    y_true- 'test' - zbiór testowy, 'train'- zbiór treningowy, lub seria danych
    '''

    mnozniki=generuj_ulamki(pr.shape[1]-1)

    merged_df = pr.copy()

    for i,col in enumerate(merged_df.drop('id',axis=1).columns.to_list()):
      merged_df[col]=mnozniki[i]*pr[col]

    if y_true=='train' :y_true = df_all[ df_all["target"].notnull() ]['target']
    elif y_true=='test' : y_true = df_all[ df_all["target"].isnull() ]['target']
    else : y_true=y_true

    y_pred= (merged_df.drop('id',axis=1).sum(axis=1) >= threshold).astype(int)
    merged_df['target']=y_pred

    if return_y_predict: return merged_df[['id','target']]
    else: return f1_score(y_true,y_pred)


### mixed_models_results()

In [18]:
def mixed_models_results(tr_predict,ts_predict,thresholds,feats=[]):
  '''
  Funkcja zwraca tabele w wynikami w formie 'model','pub','priv','diff','training'
  liczba jedynek użyta do predykcji, wyniki z Kaggle pub i priv ,
  diff- róznica między pub i priv, training -wyniki z treningu i
  walidacji na zbiorze treningowym

  Predykcja liczona łącznie ze wszystkich modeli, threshold oznacza
  liczbę jedynek w predykcjach poszczególnych modeli któe aktywują
  jedynkę w predykcji finalnej,

  Na weśjciu modele, które chcemy sprawdzić, progi , oraz nazwa klucza ze
  słownika models_trained oznaczjąca zestaw ceach które chcemy użyć
  '''
  res=[]
  models=tr_predict.columns.to_list()[:-1]
  # tr_predict=train_models_predict(models=models,feats=feats)
  # ts_predict=test_models_predict(models=models,feats=feats)
  for i,tr in enumerate(thresholds):
    f1_tr=mixed_models_predict(tr_predict,tr)
    predict=mixed_models_predict(ts_predict,tr,y_true='test',return_y_predict=True)
    pub,priv=kaggle_result(predict,df_answers)
    model_name=f'Voting :{" ".join(model[:3] for model in models)}  Jedynki :{i+1} Cechy:{feats if feats else "All"}'
    res.append((model_name,pub,priv,abs(priv-pub),f1_tr))
  sorted_res=sorted(res,key=lambda x:x[2],reverse=True)
  sorted_dif=sorted(res,key=lambda x:x[3])
  results=pd.DataFrame(res,columns=['model','pub','priv','diff','training'])
  return results

In [19]:
#tworzenie zbioru danych treningowych
def make_feats():
  black_list=['target']
  num_feats = df_all.select_dtypes("number").columns
  feats = [x for x in num_feats if x not in black_list]

  df_train = df_all[ df_all["target"].notnull() ]
  X_train = df_train[feats].values
  y_train = df_train["target"].values
  return feats, X_train,y_train

feats,X_train,y_train=make_feats()

### analyze_feature_importance()

In [20]:

def analyze_feature_importance(model, X, y,feats):
    '''
    Funkcja agreguje predykcje dla zadanego zbioru obserwacji X i danego modelu : model.
    Dla każdej klasy i dla każdej cechy zwraca zagregowane wartości wagi danej cechy :
    'min', 'max', 'mean', 'std',
    'sum_positive', 'sum_negative', - zsumowane wartości wag dodatnich i ujemnych
    'count_positive','count_negative', - liczba wystąpień wag dodatnich i ujemnych
    'mean_positive','mean_negative' - średnie wartości wag dodatnich i wag ujemnych
    '''
    feature_stats = pd.DataFrame()

    for idx, instance in tqdm(enumerate(X)):
        # Uzyskanie wyjaśnienia dla każdej obserwacji
        explanation_df = eli5.explain_prediction_df(model, instance,feature_names=feats)

        # Dodanie informacji o rzeczywistym target dla każdej obserwacji
        #explanation_df['actual_target'] = y[idx]

        # Łączenie wyników dla wszystkich obserwacji
        feature_stats = pd.concat([feature_stats, explanation_df])

    # Analiza statystyk dla każdej cechy

    feature_analysis = feature_stats.groupby(['target', 'feature'])['weight'].agg(['min', 'max', 'mean', 'std',
                                                                                   lambda x: x[x>0].sum(),
                                                                                   lambda x: x[x<0].sum(),
                                                                                   lambda x: (x>0).sum(),
                                                                                   lambda x: (x<0).sum() ])

    feature_analysis.rename(columns={'<lambda_0>': 'sum_positive', '<lambda_1>': 'sum_negative',
                                     '<lambda_2>':'count_positive','<lambda_3>':'count_negative'}, inplace=True)


    feature_analysis.reset_index(inplace=True)
    feature_analysis.drop(feature_analysis[feature_analysis['feature']=='<BIAS>'].index,axis=0,inplace=True)
    feature_analysis['mean_positive']=np.nan_to_num(feature_analysis['sum_positive'] / feature_analysis['count_positive'])
    feature_analysis['mean_negative']=np.nan_to_num(feature_analysis['sum_negative'] / feature_analysis['count_negative'])
    feature_analysis['mean_dif']=feature_analysis['mean_positive']+feature_analysis['mean_negative']
    feature_analysis=feature_analysis.round(4)


    return feature_analysis

# zmień kod w taki sposób aby średnią ustawić na zero gdy wyliczona wartość to NAN


### best_set_of_features

In [21]:
def best_set_of_features(model,feats,fia,my_range=(10,97)):
  '''
  Dla zadanego modelu i zestawieniu ważności wag ( fia) funkcja
  wybiera optymalną liczbe  cech, dla kazdej opisanej w słowniku
  feats_experiment meotdy wybory cechu z fia np:
  mean_positive_0_1 - wybierana te cechy których średnie wagi są poyztywne
  dla klasy 0 i klasy 1,  czyli otrzymamy optymalna liczbe cech dla tej metody

  Funkca zwraca słownik feats_experiment
  '''
  feats_experiment={
      'base':[],
      'count_positive_1':[],
      'mean_positive_0_and_1':[],
      'mean_positive_1':[],
      'mean_dif_0_1':[],
      'mean_dif_1':[]
  }

  feats_experiment['base'].append([len(feats),make_experiment(model, threshold=0.1,feats=feats),feats])

  for feats_num in tqdm(range(my_range[0],my_range[1]+1)):
    #count_positive >55 - połowy wystąpien jedynki
      blf=fia[(fia['target']==0)&(fia['count_positive']>55)].sort_values(by='mean',ascending=False)['feature'].to_list()[:feats_num]
      if blf: f1sc=make_experiment(model, threshold=0.1,feats=blf)
      feats_experiment['count_positive_1'].append([feats_num,f1sc,blf])
    #'mean_positive_0_and_1
      blf_1=fia[(fia['target']==1)].sort_values(by='mean_positive',ascending=False)['feature'].to_list()[:feats_num]
      blf_0=fia[(fia['target']==0)].sort_values(by='mean_positive',ascending=False)['feature'].to_list()[:feats_num]
      blf=list(set(blf_1)&set(blf_0))
      if blf: f1sc=make_experiment(model, threshold=0.1,feats=blf)
      feats_experiment['mean_positive_0_and_1'].append([feats_num,f1sc,blf])
    # 'mean_positive_1'
      blf=fia[(fia['target']==1)].sort_values(by='mean_positive',ascending=False)['feature'].to_list()[:feats_num]
      if blf: f1sc=make_experiment(model, threshold=0.1,feats=blf)
      feats_experiment['mean_positive_1'].append([feats_num,f1sc,blf])
    # 'mean_dif_0_1'
      blf_1=fia[(fia['target']==1)].sort_values(by='mean_dif',ascending=False)['feature'].to_list()[:feats_num]
      blf_0=fia[(fia['target']==0)].sort_values(by='mean_dif',ascending=False)['feature'].to_list()[:feats_num]
      blf=list(set(blf_1)&set(blf_0))
      if blf: f1sc=make_experiment(model, threshold=0.1,feats=blf)
      feats_experiment['mean_dif_0_1'].append([feats_num,f1sc,blf])
    # 'mean_dif_1'
      blf=fia[(fia['target']==1)].sort_values(by='mean_dif',ascending=False)['feature'].to_list()[:feats_num]
      if blf: f1sc=make_experiment(model, threshold=0.1,feats=blf)
      feats_experiment['mean_dif_1'].append([feats_num,f1sc,blf])

  # wybieram najlepsze wartości dla każdego klucza
  for key in feats_experiment:

     if feats_experiment[key]:
        feats_experiment[key]=max(feats_experiment[key], key=lambda x: x[1])
  return feats_experiment

In [22]:
def show_best_feats(feats_dict):
  '''
  Funckja pokazuje  formie tabeli słwonik zwracana przez funkcję best_set_of_features
  '''
  df = pd.DataFrame.from_dict(feats_dict, orient='index', columns=['Feats_count', 'F1','Feats'])
  df.reset_index(inplace=True)
  #df.drop('Cechy',axis=1,inplace=True)
  df.rename(columns={'index': 'Method'}, inplace=True)
  return df.sort_values(by='F1',ascending=False)

### create_observation_dict

In [23]:
def create_observation_dict(df, model_instance):

    """
    Funkcja wybiera obserwację z najwyższą wartością w kolumnie 'F1'
    z dataframe i tworzy słownik z danych tej obserwacji.
    Dołącza także parametry modelu i nazwę konstruktora modelu.

    :param df: DataFrame pandas z obserwacjami
    :param model: instancja wytrenowanego modelu (np. lgb.LGBMClassifier)
    :return: słownik zawierający dane z wybranej obserwacji, parametry modelu i nazwę modelu
    """
    # Wybierz obserwację z najwyższym 'F1'
    max_f1_observation = df.loc[df['F1'].idxmax()]

    # Utwórz słownik z obserwacji
    observation_dict = max_f1_observation.to_dict()

    # Dodaj do słownika parametry modelu i nazwę konstruktora
    observation_dict['params'] = model_instance.get_params()
    #observation_dict['model']=model_class

    model_dict={
            str(type(model_instance).__name__) : observation_dict}

    return model_dict

### save_models_trained

In [24]:
import json
def save_models_trained(file_name):
  '''
  Funkcja zapisuje informacje o wytrenownych modelach zebrane w słowniku
  models_trained do pliku file_name
  '''
  for k in models_trained.keys():
    if 'constructor' in models_trained[k].keys():
      del models_trained[k]['constructor']

  with open(f'../output/{file_name}', 'w') as plik_json:
    json.dump(models_trained, plik_json)



def load_models_trained(nazwa_pliku):
  '''
  Funkcja wczytuje informacje o wytrenownych modelach
  do słownika models_trained

  '''
  model_const={
      'LGBMClassifier':lgb.LGBMClassifier,
      'XGBClassifier':  xgb.XGBClassifier,
      'CatBoostClassifier':ctb.CatBoostClassifier,
      'GaussianNB':GaussianNB,
      'RandomForestClassifier':RandomForestClassifier
      }
  with open(nazwa_pliku, 'r') as plik_json:
    wczytany_slownik = json.load(plik_json)

  for key in wczytany_slownik.keys():
    if key in model_const.keys():
      wczytany_slownik[key]['constructor']=model_const[key]
    else: wczytany_slownik[key]['constructor']=''

  return wczytany_slownik

def load_models(nazwa_pliku):

  with open(nazwa_pliku, 'r') as plik_json:
    wczytany_slownik = json.load(plik_json)

  return wczytany_slownik

  models_trained.json

### ensemble_training

In [25]:
def ensemble_traininig( meta_features,y_train, meta_model=LogisticRegression(),threshold=0.1):
  '''
  funkcja za pomocą meta_modelu i cross walidacji dokonuje predykcji na podstawie predykcji modeli cząstkowych
  inaczej łączy wyniki zpredykcjiwielu modeli podanych za pomocą meeta_features przy progu aktywacji klasy 1
  równej threshold.
  '''
  predicted_probabilities = cross_val_predict(meta_model, meta_features.iloc[:,:-1], y_train, cv=5, method='predict_proba')
  predicted_labels = (predicted_probabilities[:, 1] > 0.1).astype(int)
  return f1_score(y_train,predicted_labels)



### ensemble_experiment

In [26]:

def ensemble_experiment(meta_training_features,meta_test_features,feats=[],meta_model = LogisticRegression(),threshold=0.1):
  '''
  Funkcja wykonuje predykcje dla zbioru testowego za pomocą wytrenowanego
  meta_modelu ensmble learning.


  Input:
  meta_training_features -  tabela w wynikami walidacji pojedyńczych  modeli na zbiorze treningowym
  meta_test_features = tabela z wynikami predykcji zbioru testowego dla pojedyńczych
  modeli wytrenowanych na zbiorze treningowym
  meta_model - model który połaczy predykcje
  threshold - próg aktywacji kalsy 1
  feats - nazwa klucza ze słownika models_trained

  Output:
  Funkcja zwraca dataframe z wynikami
  result=pd.DataFrame([{'model':f'Ensemble  : {" ".join(model[:3] for model in models)} Cechy:{feats}',
                       'pub':round(pub,5),
                       'priv':round(priv,5),
                       'diff':round(priv-pub,5),
                       'training':round(training_result,5)
                       }])
  '''
  models=meta_training_features.columns.to_list()[:-1]
  y_train=df_all[ df_all["target"].notnull() ]['target']


  #trening i walidacja meta modelu na danych treningowych

  training_result=ensemble_traininig(meta_training_features,y_train,threshold=threshold)

  #trening meta modelu
  meta_model.fit(meta_training_features.iloc[:,:-1],y_train)


  #predykcje zbioru testowego na meta modelu
  meta_predict=meta_model.predict_proba(meta_test_features.iloc[:,:-1])
  meta_predict_01= (meta_predict[:,1]>= threshold).astype(int)

  #wyliczanie wartości F1 dla zbioru pub i priv z Kaggle
  pub,priv=kaggle_result(pd.DataFrame({'id':meta_test_features['id'],'target':meta_predict_01}),df_answers)



  result=pd.DataFrame([{'model':f'Ensemble {meta_model.__class__.__name__[:3]} : {" ".join(model[:3] for model in models)} Cechy:{feats if feats else "All"}',
                       'pub':round(pub,5),
                       'priv':round(priv,5),
                       'diff':round(priv-pub,5),
                       'training':round(training_result,5)
                       }])
  return result

### generate_all_combinations

In [27]:


def generate_all_combinations(input_list):
    '''
    Funkcja zwraca listę wszystkihc kombinacji elementów z
    z listy input_list
    '''
    all_combinations = []
    list_length = len(input_list)
    for r in range(1, list_length + 1):
        combinations_r = list(combinations(input_list, r))
        all_combinations.extend(combinations_r)

    return all_combinations


### generate_thresholds

In [28]:
def generate_thresholds(len):
  '''
  Funkcja generuj threshold od których jest
  aktywowan klasa 1 w zależności łacznej liczby
  modeli len
  dla len =3 funkcja zwróci
  [0.33,0.33,0.34]
  '''
  i=round(1/len,2)
  thresh=[]
  for ind in range(0,len):
    thresh.append(ind*i)
  return thresh


## Wybór cech
##### wybór najlepszych cech dla kolejnych modeli. Wybór cech jest czasochłonny. Jesli cechy zostały wczesniej wybrane i modele zapisane to ten krok jest do pominięcia.

#### LGB


In [ ]:
# wszystkie cechy
model = lgb.LGBMClassifier(max_depth=5, n_estimators=100,verbosity=-1)
make_experiment(model, threshold=0.1)

0.464243955932228

In [ ]:
# Analiza ważności  cech
fia_lgb= analyze_feature_importance(model, X_train, y_train,feats)

0it [00:00, ?it/s]

zrobiłem grupowanie


In [ ]:
#wybór optymlanej liczby cech , z największym f1
feats,X_train,y_train=make_feats()
lgb_feats=best_set_of_features(lgb.LGBMClassifier(max_depth=5, n_estimators=100,verbosity=-1),
                          feats,
                          fia_lgb,
                          (10,97))


  0%|          | 0/88 [00:00<?, ?it/s]

In [ ]:
lgb_feats_df=show_best_feats(lgb_feats)

In [ ]:
# wybór metody z największym F1
models_trained.update(create_observation_dict(lgb_feats_df,model))

####XGB -1

In [ ]:
model = xgb.XGBClassifier(max_depth=5, n_estimators=100, random_state=0)
xgb_base_score=make_experiment(model, threshold=0.1)

In [ ]:
# Obliczanie ważności wag
model = xgb.XGBClassifier(max_depth=5, n_estimators=100, random_state=0)
xgb_base_score=make_experiment(model, threshold=0.1)
fia= analyze_feature_importance(model, X_train, y_train,feats)

0it [00:00, ?it/s]

zrobiłem grupowanie


In [ ]:
feats,X_train,y_train=make_feats()
xgb_feats=best_set_of_features(xgb.XGBClassifier(max_depth=5, n_estimators=100, random_state=0),
                          feats,
                          fia,
                          (10,97))

  0%|          | 0/88 [00:00<?, ?it/s]

In [ ]:
xgb_feats_df=show_best_feats(xgb_feats)

In [ ]:
xgb_feats_df

,Method,Feats_count,F1,Feats
5,mean_dif_1,42,0.479079,"[book_value_per_share_(b), interest-bearing_de..."
1,count_positive_1,59,0.477774,"[retained_earnings/total_assets, borrowing_dep..."
2,mean_positive_0_and_1,46,0.467548,"[total_assets_growth_rate_, retained_earnings/..."
3,mean_positive_1,61,0.462973,"[interest-bearing_debt_interest_rate, book_val..."
4,mean_dif_0_1,83,0.450365,"[persistent_eps_in_the_last_four_seasons, quic..."
0,base,96,0.384570,[roa(c)_before_tax_and_interest_before_depreci...


In [ ]:
models_trained.update(create_observation_dict(xgb_feats_df,model))

#### CATBOT

In [ ]:

model = ctb.CatBoostClassifier(max_depth=5, n_estimators=100, verbose=0)
ctb_base_score=make_experiment(model, threshold=0.1)
print()

# eli5.explain_prediction_df nie działa z Catboostem, niestety ni ewiem
# dlaczego zostaje przy tym co jest , spróbujemy wywołąć z cechami dla xgb i lgb


In [ ]:
for key,item in xgb_feats.items():
  cat_feats=item[2]
  model = ctb.CatBoostClassifier(max_depth=5, n_estimators=100, verbose=0)
  ctb_base_score=make_experiment(model, threshold=0.1,feats=cat_feats)
  print(f'{key}, F1:{ctb_base_score}')

In [ ]:
for key,item in lgb_feats.items():
  cat_feats=item[2]
  model = ctb.CatBoostClassifier(max_depth=5, n_estimators=100, verbose=0)
  ctb_base_score=make_experiment(model, threshold=0.1,feats=cat_feats)
  print(f'{key}, F1:{ctb_base_score}')

In [ ]:
best_ctb_features=lgb_feats['mean_positive_0_and_1'][2]

In [ ]:
models_trained.update({'CatBoostClassifier':
                       {'Method':'mean_positive_0_and_1',
                        'Feats_count':len(best_ctb_features),
                        'F1':0.49983896940418676,
                        'Feats':best_ctb_features,
                        'params':  model.get_params(),
                       }})

In [ ]:
save_models_trained('models_trained.json')

In [ ]:
models_trained.keys()

dict_keys(['LGBMClassifier', 'XGBClassifier', 'CatBoostClassifier', 'GaussianNB'])

In [ ]:
def show_bestfeats_f1():
 for key,item in models_trained.items():
   print(key,models_trained[key]['F1'])

show_bestfeats_f1()

LGBMClassifier 0.5001178272652292
XGBClassifier 0.47907907907907915
CatBoostClassifier 0.49983896940418676


#### RandomForest

In [ ]:
model = RandomForestClassifier(max_depth=5, n_estimators=100, random_state=0)
rf_base_score=make_experiment(model, threshold=0.1)
rf_base_score

0.416416244557007

In [ ]:
# Obliczanie ważności wag
model = RandomForestClassifier(max_depth=5, n_estimators=100, random_state=0)
rf_base_score=make_experiment(model, threshold=0.1)
fia= analyze_feature_importance(model, X_train, y_train,feats)

0it [00:00, ?it/s]

In [ ]:
feats,X_train,y_train=make_feats()
rf_feats=best_set_of_features(RandomForestClassifier(max_depth=5, n_estimators=100, random_state=0),
                          feats,
                          fia,
                          (10,97))

  0%|          | 0/88 [00:00<?, ?it/s]

In [ ]:
rf_feats_df=show_best_feats(rf_feats)

In [ ]:
rf_feats_df

,Method,Feats_count,F1,Feats
5,mean_dif_1,69,0.437178,"[working_capital/equity, working_capital_to_to..."
3,mean_positive_1,87,0.433849,"[working_capital/equity, working_capital_to_to..."
1,count_positive_1,44,0.432654,"[current_liability_/_liability, working_capita..."
4,mean_dif_0_1,93,0.428712,"[persistent_eps_in_the_last_four_seasons, quic..."
2,mean_positive_0_and_1,78,0.427204,"[persistent_eps_in_the_last_four_seasons, tota..."
0,base,96,0.416416,[roa(c)_before_tax_and_interest_before_depreci...


In [ ]:
save_models_trained('models_trained.json')

#### Naive Bayes


In [ ]:
feats,X_train,y_train=make_feats()
model = GaussianNB()
nb_base_score=make_experiment(model,feats=feats, threshold=0.1)
nb_base_score

0.3650555626156084

In [ ]:
models_trained.keys()

dict_keys(['LGBMClassifier', 'XGBClassifier', 'CatBoostClassifier', 'GaussianNB', 'RandomForestClassifier'])

In [ ]:
for key in models_trained.keys():
  if key!='GaussianNB':
    model = GaussianNB()
    nb_base_score=make_experiment(model, threshold=0.1,feats=models_trained[key]['Feats'])
    print(f'{key}, F1:{nb_base_score}')

LGBMClassifier, F1:0.09261833501446592
XGBClassifier, F1:0.10320982537466654
CatBoostClassifier, F1:0.12393223025440975
RandomForestClassifier, F1:0.39536875187598214


In [ ]:
feats=models_trained['RandomForestClassifier']['Feats']

In [ ]:
models_trained.update({'GaussianNB':
                       {'Method':'',
                        'Feats_count':len(feats),
                        'F1':0.39536875187598214,
                        'Feats':models_trained['RandomForestClassifier']['Feats'],
                        'params':  model.get_params(),
                       }})

In [ ]:
model = GaussianNB()
nb_base_score=make_experiment(model,feats=models_trained['GaussianNB']['Feats'], threshold=0.1,show_f1=True)

F1 score:0.39537, std:0.01132 


In [ ]:
save_models_trained('models_trained.json')

In [ ]:

for k,i in models_trained.items():


### zapis modeli

In [ ]:
save_models_trained('models_trained.json')


### WNIOSKI z wyboru cech:
##### prawidłowy wybór cech wyraźnie poprawia wyniki predykcji nawet o 9 punktów

|index|Model|Base|Selected\_Feats|improved|improved\_%|
|---|---|---|---|---|---|
|0|LGBMClassifier|0\.464|0\.5|0\.036|7\.759|
|1|XGBClassifier|0\.385|0\.479|0\.094|24\.416|
|2|CatBoostClassifier|0\.431|0\.5|0\.069|16\.009|
|3|GaussianNB|0\.365|0\.395|0\.03|8\.219|
|4|RandomForestClassifier|0\.416|0\.437|0\.021|5\.048|



In [ ]:
models_trained=load_models_trained('../output/models_trained.json')
results=pd.DataFrame(columns=['Model','Base','Selected_Feats'])
for k,i in models_trained.items():
  new_row={'Model':k,
           'Base': round(make_experiment(models_trained[k]["constructor"](**models_trained[k]["params"]),threshold=0.1),3),
           'Selected_Feats':round(models_trained[k]["F1"],3)}
  results=pd.concat([results,pd.DataFrame([new_row])],ignore_index=True)
results['improved']=round(results['Selected_Feats']-results['Base'],3)
results['improved_%']=round(results['improved']/results['Base']*100,3)
results

## Wczytaj zapisane modele

In [29]:
models_trained=load_models_trained('../output/models_trained.json')

## Optymalizacja parametrów modelu

In [ ]:
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK

In [ ]:

# Definiowanie przestrzeni hiperparametrów
model_opt_params={
    'XGBClassifier':{
            'model':xgb.XGBClassifier,
            'space':{
                    'max_depth': hp.choice('max_depth', range(3, 11)),
                    'min_child_weight': hp.quniform('min_child_weight', 1, 6, 1),
                    'subsample': hp.uniform('subsample', 0.5, 1),
                    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
                    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
                    'n_estimators': hp.choice('n_estimators', range(50, 300)),
                    'scale_pos_weight': hp.uniform('scale_pos_weight', 1, 10),
                    'eval_metric':'logloss'
                    }
            },
    'LGBMClassifier':{
            'model':lgb.LGBMClassifier,
            'space' : {
                  'max_depth': hp.choice('max_depth', range(3, 11)),
                  #'min_data_in_leaf': hp.quniform('min_data_in_leaf', 1, 100, 1),  # Zmieniono nazwę parametru
                  'min_child_samples': hp.quniform('min_child_samples', 1, 100, 1),
                  'subsample': hp.uniform('subsample', 0.5, 1),
                  'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
                  'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
                  'n_estimators': hp.choice('n_estimators', range(50, 300)),
                  'class_weight': hp.choice('class_weight', [None, 'balanced']),
                  'eval_metric':'logloss',
                  'verbose':-1}
            },
    'CatBoostClassifier':{
            'model': ctb.CatBoostClassifier,
            'space' : {
                'depth': hp.choice('depth', range(3, 11)),
                'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
                'iterations': hp.choice('iterations', range(50, 300)),
                'l2_leaf_reg': hp.uniform('l2_leaf_reg', 1, 10),
                'border_count': hp.choice('border_count', [32, 64, 128, 256]),
                'scale_pos_weight': hp.uniform('scale_pos_weight', 1, 10),
                'verbose':0}

    }
}



In [ ]:
def objective(params,model,model_feats):

    '''
    Funkcja trenuje zadany model z zadanymi parametrami,
    '''

    # Tworzenie i trenowanie modelu
    if 'min_child_samples' in params.keys():
      params['min_child_samples']=int(params['min_child_samples'])

    if 'class_weight' in params.keys():
      params['class_weight'] = int(params['class_weight'])


    classifier = model(**params)#, eval_metric='logloss')
    f1=make_experiment(classifier, threshold=0.1,feats=model_feats)

    return {'loss': -f1, 'status': STATUS_OK}

In [ ]:
%%time
### szukanie optymalnych parametrów

for key in ['XGBClassifier'] :

  model=models_trained[key]['constructor']
  space=model_opt_params[key]['space']
  model_feats=models_trained[key]['Feats']
  #X_train,X_val,y_train,y_val=make_Xy_split(models_trained[find_key(key)]['Feats'])
  trials = Trials()

  best = fmin(fn=lambda params: objective(params, model,model_feats),
            space=space,
            algo=tpe.suggest,
            max_evals=50,
            trials=trials)
  #models_trained[find_key(key)]['opt_params']=best



100%|██████████| 50/50 [01:36<00:00,  1.93s/trial, best loss: -0.47808857808857813]
CPU times: user 2min 35s, sys: 860 ms, total: 2min 36s
Wall time: 1min 36s


In [ ]:
#sprawdzenie wyników optymalizacji. model z optyamlizacją i wybranymi cechami
model=models_trained['XGBClassifier']['constructor']
model_feats=models_trained['XGBClassifier']['Feats']
classifier = model(**best)#, eval_metric='logloss')
f1=make_experiment(classifier, threshold=0.1,feats=model_feats)
print(f'XGBClassifier: {f1}')


XGBClassifier: 0.39101121743358963


In [ ]:
# model bez optymalizacji ale z wybranymi cechami
model=models_trained['XGBClassifier']['constructor']
model_feats=models_trained['XGBClassifier']['Feats']
params=models_trained['XGBClassifier']['params']
classifier = model(**params)#, eval_metric='logloss')
f1=make_experiment(classifier, threshold=0.1,feats=model_feats)
print(f'XGBClassifier: {f1}')


XGBClassifier: 0.48648648648648646


##### Wnioski z optymalizacji :
Nie ma większego sensu. Daje zbyt duże dopasowanie. I wynik na innym zbiorze danych jest gorszy o 15 punktów. Do niczego \

In [ ]:
def convert_to_json_friendly(data):

    if isinstance(data, np.int64):
        return int(data)
    elif isinstance(data, np.float64):
        return float(data)
    elif isinstance(data, list):
        return [convert_to_json_friendly(item) for item in data]
    elif isinstance(data, dict):
        return {key: convert_to_json_friendly(value) for key, value in data.items()}
    elif isinstance(data, (int, float, str, bool, type(None))):
        return data
    else:
        return str(data)
def convert_dict(dc):

  for key,item in dc.items():
      dc[key]= convert_to_json_friendly(item)
  return dc



## EKSPERYMENTY

In [30]:
#tablea łącząca wyniki eksperymentów
res_all=pd.DataFrame()

In [31]:

# predykcje dla modeli
models=['LGBMClassifier', 'XGBClassifier', 'CatBoostClassifier', 'GaussianNB', 'RandomForestClassifier']
training_predict_selected_feats=train_models_predict(models=models,feats='Feats')
training_predict_all_feats=train_models_predict(models=models)
test_predict_selected_feats=test_models_predict(models=models,feats='Feats')
test_predict_all_feats=test_models_predict(models=models)

In [32]:
# ensemble learning (LogisticRegression), na wcześniej wybranych cechach,
#  wszystkie kombinacje modeli

ensemble_results=pd.DataFrame()

for model in tqdm(generate_all_combinations(models)):
  model=list(model)
  model.append('id')

  res=ensemble_experiment( meta_training_features=training_predict_selected_feats[model],
                          meta_test_features=test_predict_selected_feats[model],
                           feats='Feats')
  ensemble_results=pd.concat([ensemble_results,res])

res_all=pd.concat([res_all,ensemble_results])

  0%|          | 0/31 [00:00<?, ?it/s]

In [33]:
# ensemble learning (LogisticRegression), na WSZYSTKICH cechach,
#  wszystkie kombinacje modeli

ensemble_results=pd.DataFrame()

for model in tqdm(generate_all_combinations(models)):
  model=list(model)
  model.append('id')

  res=ensemble_experiment( meta_training_features=training_predict_all_feats[model],
                          meta_test_features=test_predict_all_feats[model])
  ensemble_results=pd.concat([ensemble_results,res])

res_all=pd.concat([res_all,ensemble_results])

  0%|          | 0/31 [00:00<?, ?it/s]

In [34]:
# voting: wyniki predykcji modeli łączone, WYBRANE CECHY
voting_results=pd.DataFrame()

for models_combination in tqdm(generate_all_combinations(models)):
  models_combination=list(models_combination)
  thresholds=generate_thresholds(len(models_combination))
  models_combination.append('id')
  res=mixed_models_results(training_predict_selected_feats[models_combination],
                          test_predict_selected_feats[models_combination],thresholds,feats='Feats')
  voting_results=pd.concat([voting_results,res])

res_all=pd.concat([res_all,voting_results])

  0%|          | 0/31 [00:00<?, ?it/s]

In [35]:
# voting: wyniki predykcji modeli łączone, WSZYSTKIE CECHY
voting_results_all_feats=pd.DataFrame()

for models_combination in tqdm(generate_all_combinations(models)):
  models_combination=list(models_combination)

  thresholds=generate_thresholds(len(models_combination))
  models_combination.append('id')
  #print(f'tr:{thresholds} , długość:{len(models_combination)}')
  res=mixed_models_results(training_predict_all_feats[models_combination],
                          test_predict_all_feats[models_combination],thresholds)
  voting_results_all_feats=pd.concat([voting_results_all_feats,res])

res_all=pd.concat([res_all,voting_results_all_feats])

  0%|          | 0/31 [00:00<?, ?it/s]

In [41]:
res_all=res_all.round(5)

In [42]:
#NAJLEPSZY wynik po priv - wynik na kaggle private bord
res_all.sort_values(by='priv',ascending=False).head(5)

,model,pub,priv,diff,training
2,Voting :LGB XGB Cat Gau Jedynki :3 Cechy:Feats,0.40000,0.54167,0.14167,0.50000
2,Voting :LGB XGB Cat Gau Jedynki :3 Cechy:All,0.44275,0.52632,0.08357,0.46094
0,Ensemble Log : LGB XGB Cat Gau Cechy:Feats,0.46032,0.51128,0.05096,0.49805
3,Voting :LGB XGB Cat Ran Jedynki :4 Cechy:All,0.39669,0.50877,0.11208,0.45946
2,Voting :LGB XGB Gau Jedynki :3 Cechy:All,0.35514,0.50847,0.15333,0.45815


In [48]:
res_all.sort_values(by='priv',ascending=False).head(10)

,model,pub,priv,diff,training
2,Voting :LGB XGB Cat Gau Jedynki :3 Cechy:Feats,0.40000,0.54167,0.14167,0.50000
2,Voting :LGB XGB Cat Gau Jedynki :3 Cechy:All,0.44275,0.52632,0.08357,0.46094
0,Ensemble Log : LGB XGB Cat Gau Cechy:Feats,0.46032,0.51128,0.05096,0.49805
3,Voting :LGB XGB Cat Ran Jedynki :4 Cechy:All,0.39669,0.50877,0.11208,0.45946
2,Voting :LGB XGB Gau Jedynki :3 Cechy:All,0.35514,0.50847,0.15333,0.45815
2,Voting :XGB Gau Ran Jedynki :3 Cechy:All,0.34722,0.50382,0.15659,0.42907
2,Voting :LGB Cat Ran Jedynki :3 Cechy:Feats,0.43056,0.49315,0.06260,0.50360
0,Ensemble Log : LGB Cat Gau Ran Cechy:Feats,0.43056,0.49315,0.06260,0.44872
0,Ensemble Log : LGB Cat Ran Cechy:Feats,0.43056,0.49315,0.06260,0.48443
0,Ensemble Log : Cat Gau Cechy:All,0.42759,0.49296,0.06537,0.43123


In [38]:
#Najlepsze wyniki po walidaccji na zbiorze treningowym
#całkim niezłe wyniki, pierwszy i srugi wynik daje mi 1 miejsce na Kaggleu
#drugi model jest super stabliny.Daje tak samo dobre wyniki na pub jak i na priv!

res_all.sort_values(by='training',ascending=False).head(5)

,model,pub,priv,diff,training
3,Voting :LGB XGB Cat Gau Ran Jedynki :4 Cechy:...,0.455882,0.488189,0.032307,0.523438
2,Voting :XGB Cat Gau Jedynki :3 Cechy:Feats,0.486957,0.488550,0.001593,0.516393
3,Voting :XGB Cat Gau Ran Jedynki :4 Cechy:Feats,0.520325,0.436975,0.083350,0.512605
2,Voting :LGB XGB Ran Jedynki :3 Cechy:Feats,0.470588,0.449612,0.020976,0.512000
3,Voting :LGB XGB Cat Gau Jedynki :4 Cechy:Feats,0.514286,0.403509,0.110777,0.509259


In [47]:
res_all[res_all['training']>=0.5].sort_values(by='training',ascending=False)

,model,pub,priv,diff,training
3,Voting :LGB XGB Cat Gau Ran Jedynki :4 Cechy:...,0.45588,0.48819,0.03231,0.52344
2,Voting :XGB Cat Gau Jedynki :3 Cechy:Feats,0.48696,0.48855,0.00159,0.51639
3,Voting :XGB Cat Gau Ran Jedynki :4 Cechy:Feats,0.52033,0.43697,0.08335,0.51261
2,Voting :LGB XGB Ran Jedynki :3 Cechy:Feats,0.47059,0.44961,0.02098,0.51200
3,Voting :LGB XGB Cat Gau Jedynki :4 Cechy:Feats,0.51429,0.40351,0.11078,0.50926
3,Voting :LGB XGB Cat Ran Jedynki :4 Cechy:Feats,0.53125,0.40351,0.12774,0.50877
2,Voting :LGB XGB Gau Jedynki :3 Cechy:Feats,0.52632,0.36667,0.15965,0.50862
3,Voting :LGB Cat Gau Ran Jedynki :4 Cechy:Feats,0.51562,0.41667,0.09896,0.50820
0,Ensemble Log : LGB XGB Gau Cechy:Feats,0.52632,0.36667,-0.15965,0.50644
2,Voting :LGB Cat Ran Jedynki :3 Cechy:Feats,0.43056,0.49315,0.06260,0.50360


In [51]:
#szukam najstabilniejszych modeli czyli z najmnijeszą różnica miedzy pub priv i training

res_all['diff_mean']=(abs(res_all['pub']-res_all['priv'])+abs(res_all['pub']-res_all['training'])+abs(res_all['priv']-res_all['training']))/3

In [52]:
res_all.sort_values(by='diff_mean',ascending=True).head(5)

,model,pub,priv,diff,training,diff_mean
1,Voting :XGB Ran Jedynki :2 Cechy:All,0.41081,0.41111,0.00030,0.40838,0.001820
2,Voting :LGB XGB Gau Ran Jedynki :3 Cechy:All,0.44000,0.43662,0.00338,0.44000,0.002253
0,Voting :LGB Cat Jedynki :1 Cechy:All,0.06580,0.05919,0.00661,0.06252,0.004407
0,Voting :LGB XGB Cat Jedynki :1 Cechy:Feats,0.06580,0.05919,0.00661,0.06252,0.004407
0,Voting :LGB XGB Gau Jedynki :1 Cechy:Feats,0.06580,0.05919,0.00661,0.06252,0.004407


In [53]:
# Pojedyńcze modele - zestawienie danych

models=['LGBMClassifier', 'XGBClassifier', 'CatBoostClassifier', 'GaussianNB', 'RandomForestClassifier']

#wyszukuję wyników dla pojedyńczych modeli
base_models_results=res_all.sort_values(by='model',key=lambda x:x.str.len()).head(10)
base_models_results.drop('diff_mean',axis=1,inplace=True)

#dziele na modele przeliczone dla wybranych cech i wszystkich cech
base_models_results_all=base_models_results.iloc[:len(models),:].copy()
base_models_results_Feats=base_models_results.iloc[len(models):,:].copy()

#ustawiam nazwy modeli
base_models_results_Feats['model']=models
base_models_results_all['model']=models

#dodaje sufiksy do nazw kolumn all- wszystkice cechy Feats- wyselekcjonowane
base_models_results_all.columns=[x + '_all' for x in base_models_results_all.columns]
base_models_results_Feats.columns=[x + '_Feats' for x in base_models_results_Feats.columns]

#łącze powyższe ze sobą
base_models_merged=base_models_results_all.merge(base_models_results_Feats,left_on='model_all',right_on='model_Feats')

#sortuje kolumny w taki sposob aby kolumny o podobnych nazwach były obok siebie
sorted_columns = sorted(base_models_merged.columns, key=lambda x: x.split('_')[0])
base_models_sorted =base_models_merged[sorted_columns]

#usuwam niepotrzebne kolumny z nazwami modeli i zmieniam nazwe pierwszej na model
base_models_sorted=base_models_sorted.drop(base_models_sorted.columns[:3],axis=1)
base_models_sorted.rename(columns={base_models_sorted.columns[0]: 'model'}, inplace=True)

#licze F1 dla wszystkich pojedyńczych modeli z progiem aktywacji klasy 1 na poziomie 0.5 (klasycznie)
training_05=pd.DataFrame()
for model in models:
  m=models_trained[model]['constructor'](**models_trained[model]['params'])
  training_05=pd.concat([training_05,
                         pd.DataFrame([{'model':model,'training_05':make_experiment(m)}])])
base_models_sorted=base_models_sorted.merge(training_05,on='model')

#dodaje kolumny: training best- z najlepszym wynikiem waalidacji na zbiorze treningowym
#oraz pirv_best - najlepszym wynikiem na zbiorze testowym
base_models_sorted['training_best']=res_all.sort_values(by='training',ascending=False).head(1)['training'].values[0]
base_models_sorted['priv_best']=res_all.sort_values(by='priv',ascending=False).head(1)['priv'].values[0]

In [56]:
base_models_sorted=base_models_sorted.round(5)

In [57]:
#tabelka z zestawieniem wyników (f1) na zbiorze testowym
#priv_all - wszystkich cechy, priv_Feats - wybrane cechy
#priv_best - wyniki najlepszego modelu ensemble

base_models_priv=base_models_sorted[[*base_models_sorted.columns[:3],
                       base_models_sorted.columns[-1]]]
base_models_priv

,model,priv_all,priv_Feats,priv_best
0,LGBMClassifier,0.44444,0.44275,0.54167
1,XGBClassifier,0.49296,0.44643,0.54167
2,CatBoostClassifier,0.26357,0.46479,0.54167
3,GaussianNB,0.43478,0.36641,0.54167
4,RandomForestClassifier,0.33929,0.34524,0.54167


In [63]:
#tabelka z zestawieniem wyników (f1) dla pojedyńczych modeli
#priv,pub - to wyniki f1 na zbiorze testowym
#training - wyniki walidacji na zbiorze treningowym
#all - wszystkie cechy
#Feats - wybrane cechy
#best - najlepszy model ensemble
base_models_pub_priv=base_models_sorted[['model',
 'priv_all',
 'pub_all',
 'training_all',
 'priv_Feats',
 'pub_Feats',
 'training_Feats',
 'priv_best',
 'training_best',
 ]]
base_models_pub_priv

,model,priv_all,pub_all,training_all,priv_Feats,pub_Feats,training_Feats,priv_best,training_best
0,LGBMClassifier,0.44444,0.46281,0.46552,0.44275,0.47934,0.50213,0.54167,0.52344
1,XGBClassifier,0.49296,0.42759,0.43123,0.44643,0.46400,0.47826,0.54167,0.52344
2,CatBoostClassifier,0.26357,0.46377,0.36242,0.46479,0.46043,0.48227,0.54167,0.52344
3,GaussianNB,0.43478,0.40000,0.41622,0.36641,0.38235,0.39590,0.54167,0.52344
4,RandomForestClassifier,0.33929,0.48649,0.38356,0.34524,0.49462,0.43767,0.54167,0.52344


In [55]:
#tableka z wynikami walidacji pojedyńczych modeli na zbiorze treningowym
# trainin_05- threshold aktywacji klasy 1 równy 0.5, wszystkie cechy
# training_all - threshold aktywacji klasy 1 równy 0.1 wszystkie cechy
# training_Feats - threshold aktwyacji klasy 1 równy 0.1 wybrane cechy
#trainin_best - wynik najlepszego modelu ensemble

base_models_train=base_models_sorted[[base_models_sorted.columns[:2],
                       base_models_sorted.columns[3],
                       *base_models_sorted.columns[5:7],
                       base_models_sorted.columns[-2]]]
base_models_train

,model,training_05,training_all,training_Feats,training_best
0,LGBMClassifier,0.277980,0.46552,0.50213,0.52344
1,XGBClassifier,0.306452,0.43123,0.47826,0.52344
2,CatBoostClassifier,0.219093,0.36242,0.48227,0.52344
3,GaussianNB,0.354719,0.41622,0.39590,0.52344
4,RandomForestClassifier,0.174286,0.38356,0.43767,0.52344


## WNIOSKI
#Najlepsze modele:

### Model z najlepszym wynikiem dla priv (konkurs Kaggle)

|index|model|pub|priv|diff|training|
|---|---|---|---|---|---|
|2|Voting :LGB XGB Cat Gau  Jedynki :3 Cechy:Feats|0\.4|0\.54167|0\.14167|0\.5|

  Problem z tym modelem polaga na tym, że najprawdopodobniej nie wybrałbym go ze względu na nie najwyższą wartość walidacji na danych treningowych (training). Jest 14 modeli z lepszym wynikiem walidacji.

### Modele z najlepszymi wynikami walidacji:

|index|model|pub|priv|diff|training|
|---|---|---|---|---|---|
|3|Voting :LGB XGB Cat Gau Ran  Jedynki :4 Cechy:Feats|0\.45588|0\.48819|0\.03231|0\.52344|
|2|Voting :XGB Cat Gau  Jedynki :3 Cechy:Feats|0\.48696|0\.48855|0\.00159|0\.51639|
|3|Voting :XGB Cat Gau Ran  Jedynki :4 Cechy:Feats|0\.52033|0\.43697|0\.08335|0\.51261|

Dwa pierwsze zgłosiłbym do konkursu ze względu na najwyższe wartości walidacji (training). Obydwa dają pierwsze miejsce ( wynik do pobicia to 0.48695).
Drugi model jest bardzo stabilny daje bardzo wysokie wyniki dla całego zbioru testowego: pub i priv




#Co wpłyneło na wynik ?

Poniżej tabela pokazująca jak zmieniją się wyniki walidacji dla poszcególnych modeli wraz z kolejnymi krokami.

Opis tabelki

**training_05**- threshold aktywacji klasy 1 równy 0.5, wszystkie cechy,

**training_all** - threshold aktywacji klasy 1 równy 0.1,wszystkie cechy,

**training_Feats** - threshold aktwyacji klasy 1 równy 0.1, wybrane cechy,

**trainin_best** - wynik najlepszego modelu ensemble:

|index|model|training\_05|training\_all|training\_Feats|training\_best|
|---|---|---|---|---|---|
|0|LGBMClassifier|0\.277979797979798|0\.46552|0\.50213|0\.52344|
|1|XGBClassifier|0\.3064516129032258|0\.43123|0\.47826|0\.52344|
|2|CatBoostClassifier|0\.21909331108329858|0\.36242|0\.48227|0\.52344|
|3|GaussianNB|0\.35471883843976865|0\.41622|0\.3959|0\.52344|
|4|RandomForestClassifier|0\.1742857142857143|0\.38356|0\.43767|0\.52344|






  1. Dobór właściwego progu aktywacji klasy 1 kluczowy. Tutaj zszedłem do 0.1 (z 0.5) co poprawiło wyniki nawet o 21 punktów (RandomForrest).
  2. Dobór właściwych cech to kolejne 4-5 punktów (12 w przypadku CatBoost)
  3. Łączenie wyników to kolejne 4-5 punktów. Metoda głoswania wydaje się odrobinę skuteczniejsza, od łączenia wyników za pomocą metamodelu. Być może jest to kwestia wyboru metamodelu, lub jego dostrojenia.
  4. Optymalizacja parametrów nie dała żadnych pozytwnych wyników

#Co dalej?
1.Zbadać SMOTE - balansowania danych. Tutaj klasa 1 jest wyrażnie mniej liczna.Tylko ok 3,2%

2.Spróbować grupowania danych.Być może są grupy gdzie liczność klasy 1 jest wyraźnie wyższa.

3.Wypróbować i dodać inne algorytmy : SVM, sieci neuronowe...

4.Zbadać i zrozumieć jak poszcególne algorytmy wpływają ną łączny wynik (XGB jest w każdym z najlepszych modeli, Randomorest już niekoniecznie)

#Czego nie rozumiem:
1. Jak wybrać najlepszy model - taki który dam nam najlepszy wynik na priv?

